<a href="https://colab.research.google.com/github/SecurityCyberGeek/MAS-DSE-LLM_Student_Improvement/blob/main/Math_3B_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone - Using LLMs for Student Outcome

This function modified Vocareum Math 3B data to correlate result from an xlms file.

### ERRORS:
1) 'utf-8' codec can't decode byte 0x97 in position 809: invalid start byte
- Possible that Feature has strings in it. Maybe char "\" followed by non-escapable char (like \(ax = b\), which is not UTF-8
2) Look at which char (0x97) is the culprit. The character 0x97 is often the Windows smart dash (– or —) — a long dash character from Windows-1252 (a.k.a. CP1252 encoding), not valid in UTF-8.
3) Solution: Try specifying encoding='cp1252' or encoding='ISO-8859-1'
- E.g., data_conversations = pd.read_csv(file_path_conversations, encoding='cp1252')


## Prepare Environment for Import
Use Google Collab to work on the environment with multiple developers.

### Imports

In [ ]:
import pandas as pd
import enum
import numpy as np
import string
import random

### Connect to Google Drive (change to local drive if using local Jupyter Notebook)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### For shared Google Drive:
- Ensure the following variables are in an accessible location
- This will be the owner's local drive, or another developers 'shared' drive.

In [ ]:
# enum  LOCATION = {
#     LOCAL = '/content/drive/My Drive/DSE_260_Capstone/Math 3B/',
#     SHARED = '/content/drive/Shared drives/DSE 260 Capstone/Math 3B'
# }

### Mount Input

- A user may not be able to access a folder from Google Drive (i.e., DSE 260 Capstone)
- E.g., "FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/DSE_260_Capstone/Charts for Students - Normal MVP Data Set.csv'"


- If it cannot be accessed:
1) Ensure that it is saved as a CSV (Export to CSV in Excel or Open Office)
2) Find the file in the shared drive, right-click => Organize => create a shortcut.
3) Then paste the shortcut to your personal drive (My Drive (main), no other folder).

4) ** NOTE IF THERE ARE UNDERSCORES IN THE NAME - The working here has underscores**

In [ ]:
#file_path = '/content/drive/MyDrive/DSE 260 Capstone/course_grades_vc_2_0_08172024_CSS1.csv'
#file_path = '/content/drive/MyDrive/DSE_260_Capstone/course_grades_vc_2_0_08172024 - CSS1 - Truncated.csv' # Shared User: Add shortcut. Ensure/Add underlines in name if necessary.
#file_path = '/content/drive/My Drive/DSE_260_Capstone/Math 3B/Math 3B Anonymized Gradebook - Sheet2.xlsx' # Shared User: Add shortcut. Ensure/Add underlines in name if necessary.
file_path = '/content/drive/My Drive/Test CSV.csv' # Shared User: Add shortcut. Ensure/Add underlines in name if necessary.

file_path = '/content/drive/My Drive/DSE_260_Capstone/Math 3B/Math 3B Anonymized Gradebook.csv' #course_grades_vc_2_0_08172024 - CSS1 - Truncated.csv'
data = pd.read_csv(file_path)
data.head()

,Student #,Vocareum RandomUser#,Week 2 HW - 1.1 Precalc 2e,Week 2 HW - 1.2 Precalc 2e,Week 3 HW - 1.3 Precalc 2e,Week 3 HW - 1.5 Precalc 2e,Week 4 HW - 2.1 Precalc 2e,Week 4 HW - 2.2 Precalc 2e,Week 4 HW - 2.3 Precalc 2e,Week 5 HW - 1.5 CA / 2.5 P
0,From Ko,From Voc,1.1,1.2,1.3,1.5,2.1,2.2,2.3,1.5/2.5
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18
3,3,NaN,10/10,9/9,Late,Late,Late,Late,Late,Late
4,4,178739370,10/10,9/9,9/9,10/10,6/6,8/8,6/7,18/18


# DATA CLEANING - ANONYMIZED GRADEBOOK
- CSV is Math 3B Anonymized Gradebook.csv
- Used to compare AI users vs. Non-AI Users
- Only grades Homeworks (Weeks 1-5). Tests (HW1-4) and Midterm cleaned later in code.

Cleaning:
- Remove NaNs
- Ensure top line is header
- Scores are numeric, "Lates" are averages
- Extract AI vs. non-AI
- Change features to shorter names
- Change RandomUser# to Vocareum ID
- Add (Total HW) Score, GPA %, Letter Grade
- Export to CSV

In [ ]:
copy_df = data.copy()

In [ ]:
copy_df.head(3)

,Student #,Vocareum RandomUser#,Week 2 HW - 1.1 Precalc 2e,Week 2 HW - 1.2 Precalc 2e,Week 3 HW - 1.3 Precalc 2e,Week 3 HW - 1.5 Precalc 2e,Week 4 HW - 2.1 Precalc 2e,Week 4 HW - 2.2 Precalc 2e,Week 4 HW - 2.3 Precalc 2e,Week 5 HW - 1.5 CA / 2.5 P
0,From Ko,From Voc,1.1,1.2,1.3,1.5,2.1,2.2,2.3,1.5/2.5
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18


In [ ]:
singleHeader_df = copy_df.iloc[1:]  # Remove the top row (From Ko, From Voc, etc.). Starts AFTER index 0 and later...

In [ ]:
singleHeader_df.head(3)

,Student #,Vocareum RandomUser#,Week 2 HW - 1.1 Precalc 2e,Week 2 HW - 1.2 Precalc 2e,Week 3 HW - 1.3 Precalc 2e,Week 3 HW - 1.5 Precalc 2e,Week 4 HW - 2.1 Precalc 2e,Week 4 HW - 2.2 Precalc 2e,Week 4 HW - 2.3 Precalc 2e,Week 5 HW - 1.5 CA / 2.5 P
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18
3,3,NaN,10/10,9/9,Late,Late,Late,Late,Late,Late


In [ ]:
shortTitle_df = singleHeader_df.rename(columns={
    'Student #': 'Student #',
    'Vocareum RandomUser#': 'Vocareum ID',
    'Week 2 HW - 1.1 Precalc 2e': 'HW 1.1',
    'Week 2 HW - 1.2 Precalc 2e': 'HW 1.2',
    'Week 3 HW - 1.3 Precalc 2e': 'HW 1.3',
    'Week 3 HW - 1.5 Precalc 2e': 'HW 1.4',
    'Week 4 HW - 2.1 Precalc 2e': 'HW 2.1',
    'Week 4 HW - 2.2 Precalc 2e': 'HW 2.2',
    'Week 4 HW - 2.3 Precalc 2e': 'HW 2.3',
    'Week 5 HW - 1.5 CA / 2.5 P': 'HW 2.5'
    })

In [ ]:
shortTitle_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18
3,3,NaN,10/10,9/9,Late,Late,Late,Late,Late,Late


In [ ]:
shortTitle_df.count()

,0
Student #,140
Vocareum ID,81
HW 1.1,139
HW 1.2,139
HW 1.3,139
HW 1.4,139
HW 2.1,139
HW 2.2,139
HW 2.3,139
HW 2.5,139


## Filter AI Users vs. Non-AI Users
-  Identify rows containing NaN values using the isna() method in combination with any(axis=1).
- This creates a boolean mask where True indicates a row with at least one NaN.

In [ ]:
noAI_mask = shortTitle_df.isna().any(axis=1)

In [ ]:
# Then, use this mask to select the desired rows.
noAI_df = shortTitle_df[noAI_mask]

In [ ]:
noAI_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18
3,3,NaN,10/10,9/9,Late,Late,Late,Late,Late,Late
8,8,NaN,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18


In [ ]:
noAI_df.count()

,0
Student #,59
Vocareum ID,0
HW 1.1,58
HW 1.2,58
HW 1.3,58
HW 1.4,58
HW 2.1,58
HW 2.2,58
HW 2.3,58
HW 2.5,58


- We assign Vocareum IDs (The Random #'s that relate to other charts) with AI users.
- We assign students with no Vocareum IDs with manual entries only (to compare grades)

### Now to filter for the Vocareum Users

In [ ]:
noNaN_df = shortTitle_df.dropna()

In [ ]:
noNaN_df.head(10)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
4,4,178739370,10/10,9/9,9/9,10/10,6/6,8/8,6/7,18/18
5,5,179357400,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
6,6,178739730,10/10,9/9,8/9,8/10,6/6,8/8,7/7,18/18
7,7,179066835,10/10,9/9,9/9,9/10,6/6,7/8,7/7,17/18
9,9,178739415,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
10,10,178739595,10/10,9/9,9/9,10/10,4/6,Late,Late,18/18
11,11,179457750,10/10,Late,9/9,10/10,6/6,8/8,7/7,18/18
12,12,179072505,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
14,14,178740045,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18


In [ ]:
noNaN_df.count()

,0
Student #,81
Vocareum ID,81
HW 1.1,81
HW 1.2,81
HW 1.3,81
HW 1.4,81
HW 2.1,81
HW 2.2,81
HW 2.3,81
HW 2.5,81


### Use ApplyMap to Remove "Late" cells
- The code calculates the mean of each column using df.mean().
- Then, it employs applymap to iterate through each cell, replacing it with the corresponding column's mean.

In [ ]:
ai_user_noLate_df = pd.DataFrame(noNaN_df)

def remove_late(cell):
    if isinstance(cell, str) and 'Late' in cell:
        return cell.replace('Late', '').strip()
    return cell

ai_user_noLate_df[['HW 1.1',	'HW 1.2', 'HW 1.3',
          'HW 1.4','HW 2.1',	'HW 2.2',
          'HW 2.3',	'HW 2.5']] = noNaN_df[['HW 1.1',	'HW 1.2', 'HW 1.3',
                                           'HW 1.4','HW 2.1',	'HW 2.2',
                                           'HW 2.3',	'HW 2.5']].applymap(remove_late)


<ipython-input-309-728976e74cf1>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'HW 2.3',	'HW 2.5']].applymap(remove_late)


In [ ]:
ai_user_noLate_df.head(10)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
1,1,178739325,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
4,4,178739370,10/10,9/9,9/9,10/10,6/6,8/8,6/7,18/18
5,5,179357400,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
6,6,178739730,10/10,9/9,8/9,8/10,6/6,8/8,7/7,18/18
7,7,179066835,10/10,9/9,9/9,9/10,6/6,7/8,7/7,17/18
9,9,178739415,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
10,10,178739595,10/10,9/9,9/9,10/10,4/6,,,18/18
11,11,179457750,10/10,,9/9,10/10,6/6,8/8,7/7,18/18
12,12,179072505,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18
14,14,178740045,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18


### Use the same function for the 'no_ai' students
- Remember to copy the prior dataframe (to keep Student # and Vocareum ID)

In [ ]:
no_ai_noLate_df = pd.DataFrame(noAI_df)

no_ai_noLate_df[['HW 1.1',	'HW 1.2', 'HW 1.3',
          'HW 1.4','HW 2.1',	'HW 2.2',
          'HW 2.3',	'HW 2.5']] = noAI_df[['HW 1.1',	'HW 1.2', 'HW 1.3',
                                           'HW 1.4','HW 2.1',	'HW 2.2',
                                           'HW 2.3',	'HW 2.5']].applymap(remove_late)

<ipython-input-311-72bde9b49c80>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'HW 2.3',	'HW 2.5']].applymap(remove_late)


In [ ]:
no_ai_noLate_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
2,2,NaN,10/10,9/9,9/9,8/10,6/6,8/8,6/7,18/18
3,3,NaN,10/10,9/9,,,,,,
8,8,NaN,10/10,9/9,9/9,10/10,6/6,8/8,7/7,18/18


## Convert Fractional Scores into single numbers (strings)
- In order to fill in the blanks, we need numbers to calculate the mean
- Therefore, remove the '/' between the numbers

In [ ]:
# Remove everything after the last forwardslash

# First, copy to a modifiable df
ai_scores_and_blanks = pd.DataFrame(ai_user_noLate_df)

'''Something like this will work.  However, only works for ONE column, not multiple:
# Replace blank cells with NaN
df.replace('', np.nan, inplace=True)

# Calculate the mean of each column, skipping NaN values
column_means = df.mean(numeric_only=True)

# Replace NaN values with the mean of their respective columns
df = df.fillna(column_means)
'''

cols_to_clean = ['HW 1.1',	'HW 1.2', 'HW 1.3',
          'HW 1.4','HW 2.1',	'HW 2.2',
          'HW 2.3',	'HW 2.5']  # Add more as needed

for col in cols_to_clean:
    ai_scores_and_blanks[col] = ai_scores_and_blanks[col].str.split('/').str[0]

In [ ]:
ai_scores_and_blanks.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
1,1,178739325,10,9,9,10,6,8,7,18
4,4,178739370,10,9,9,10,6,8,6,18
5,5,179357400,10,9,9,10,6,8,7,18


In [ ]:
# Same for the Non-AI students

# First, copy to a modifiable df
no_ai_scores_and_blanks = pd.DataFrame(no_ai_noLate_df)

for col in cols_to_clean:
    no_ai_scores_and_blanks[col] = no_ai_scores_and_blanks[col].str.split('/').str[0]

no_ai_scores_and_blanks.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
2,2,NaN,10,9,9,8,6,8,6,18
3,3,NaN,10,9,,,,,,
8,8,NaN,10,9,9,10,6,8,7,18


In [ ]:
ai_scores_and_blanks.dtypes

,0
Student #,object
Vocareum ID,object
HW 1.1,object
HW 1.2,object
HW 1.3,object
HW 1.4,object
HW 2.1,object
HW 2.2,object
HW 2.3,object
HW 2.5,object


### We must convert these Strings to Ints

In [ ]:
assignment_columns = ['HW 1.1',	'HW 1.2', 'HW 1.3',
          'HW 1.4','HW 2.1',	'HW 2.2',
          'HW 2.3',	'HW 2.5']

ai_scores_and_blanks[assignment_columns] = ai_scores_and_blanks[assignment_columns].apply(pd.to_numeric, errors='coerce') # Converts to int if not None, else replaces with itself (should be blank)

In [ ]:
ai_scores_and_blanks.dtypes

,0
Student #,object
Vocareum ID,object
HW 1.1,float64
HW 1.2,float64
HW 1.3,float64
HW 1.4,float64
HW 2.1,float64
HW 2.2,float64
HW 2.3,float64
HW 2.5,float64


In [ ]:
no_ai_scores_and_blanks[assignment_columns] = no_ai_scores_and_blanks[assignment_columns].apply(pd.to_numeric, errors='coerce') # Converts to int if not None, else replaces with itself (should be blank)

### Now instead, we want the means in place of the blank cells

### First the AI Users

In [ ]:
ai_scores = pd.DataFrame(ai_scores_and_blanks)
# Fill blanks with the column average
ai_scores[assignment_columns] = ai_scores[assignment_columns].fillna(
    ai_scores[assignment_columns].mean()
)

In [ ]:
ai_scores.head(10)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
1,1,178739325,10.0,9.000000,9.0,10.0,6.0,8.000000,7.000000,18.0
4,4,178739370,10.0,9.000000,9.0,10.0,6.0,8.000000,6.000000,18.0
5,5,179357400,10.0,9.000000,9.0,10.0,6.0,8.000000,7.000000,18.0
6,6,178739730,10.0,9.000000,8.0,8.0,6.0,8.000000,7.000000,18.0
7,7,179066835,10.0,9.000000,9.0,9.0,6.0,7.000000,7.000000,17.0
9,9,178739415,10.0,9.000000,9.0,10.0,6.0,8.000000,7.000000,18.0
10,10,178739595,10.0,9.000000,9.0,10.0,4.0,7.831169,6.945946,18.0
11,11,179457750,10.0,8.985915,9.0,10.0,6.0,8.000000,7.000000,18.0
12,12,179072505,10.0,9.000000,9.0,10.0,6.0,8.000000,7.000000,18.0
14,14,178740045,10.0,9.000000,9.0,10.0,6.0,8.000000,7.000000,18.0


In [ ]:
no_ai_scores = pd.DataFrame(no_ai_scores_and_blanks)

# Fill blanks with the column average
no_ai_scores[assignment_columns] = no_ai_scores[assignment_columns].fillna(
    no_ai_scores[assignment_columns].mean()
)

In [ ]:
no_ai_scores.head(10)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5
2,2,NaN,10.0,9.0,9.000000,8.000000,6.000000,8.00,6.000000,18.00
3,3,NaN,10.0,9.0,8.653061,9.530612,5.958333,7.92,6.938776,17.16
8,8,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00
13,13,NaN,10.0,9.0,9.000000,9.000000,6.000000,8.00,7.000000,14.00
15,15,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00
16,16,NaN,10.0,9.0,9.000000,7.000000,6.000000,7.00,6.938776,17.16
19,19,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00
21,21,NaN,10.0,9.0,9.000000,10.000000,5.000000,8.00,7.000000,17.00
27,27,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00
28,28,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00


In [ ]:
# Calculate scores based on columns_to_assign, provide result to nearest 10th
def calculate_total_points(dataframe, columns_to_assign):
    # Calculate Final_Score using Excel formula
    base_scores = (dataframe[columns_to_assign].sum(axis=1)).round(1)
    return base_scores

In [ ]:
ai_total_scores = pd.DataFrame(ai_scores)
ai_total_scores["HW Score"] = calculate_total_points(ai_scores, assignment_columns)
ai_total_scores.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score
1,1,178739325,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0
4,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,76.0
5,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0


In [ ]:
no_ai_total_scores = pd.DataFrame(no_ai_scores)
no_ai_total_scores["HW Score"] = calculate_total_points(no_ai_scores, assignment_columns)
no_ai_total_scores.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score
2,2,NaN,10.0,9.0,9.000000,8.000000,6.000000,8.00,6.000000,18.00,74.0
3,3,NaN,10.0,9.0,8.653061,9.530612,5.958333,7.92,6.938776,17.16,75.2
8,8,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00,77.0


## Calcuate GPA (as Float out of 100)

In [ ]:
ai_gpa = pd.DataFrame(ai_total_scores)
ai_gpa["HW GPA_Percent"] = (ai_gpa["HW Score"] / (10+9+9+10+6+8+7+18) * 100).round(1)
ai_gpa.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent
1,1,178739325,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0,100.0
4,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,76.0,98.7
5,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0,100.0


In [ ]:
no_ai_gpa = pd.DataFrame(no_ai_total_scores)
no_ai_gpa["HW GPA_Percent"] = (no_ai_gpa["HW Score"] / (10+9+9+10+6+8+7+18) * 100).round(1)
no_ai_gpa.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent
2,2,NaN,10.0,9.0,9.000000,8.000000,6.000000,8.00,6.000000,18.00,74.0,96.1
3,3,NaN,10.0,9.0,8.653061,9.530612,5.958333,7.92,6.938776,17.16,75.2,97.7
8,8,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00,77.0,100.0


## Calculate Letter Grade

In [ ]:
# Assign letter grades with verification of distribution
def assign_letter_grade(percent):
    percent = round(percent, 2)
    if percent >= 90:
        letter = 'A'
        remainder = round(percent - 90, 2)
    elif percent >= 80:
        letter = 'B'
        remainder = round(percent - 80, 2)
    elif percent >= 70:
        letter = 'C'
        remainder = round(percent - 70, 2)
    elif percent >= 60:
        letter = 'D'
        remainder = round(percent - 60, 2)
    else:
        return 'F'

    modifier = '+' if remainder >= 7 else ('-' if remainder < 3 else '')
    return letter + modifier

In [ ]:
ai_final_df = pd.DataFrame(ai_gpa)
ai_final_df['HW Letter Grade'] = ai_final_df['HW GPA_Percent'].apply(assign_letter_grade)
ai_final_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW Letter Grade
1,1,178739325,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0,100.0,A+
4,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,76.0,98.7,A+
5,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0,100.0,A+


In [ ]:
no_ai_final_df = pd.DataFrame(no_ai_gpa)
no_ai_final_df['HW Letter Grade'] = no_ai_final_df['HW GPA_Percent'].apply(assign_letter_grade)
no_ai_final_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW Letter Grade
2,2,NaN,10.0,9.0,9.000000,8.000000,6.000000,8.00,6.000000,18.00,74.0,96.1,A
3,3,NaN,10.0,9.0,8.653061,9.530612,5.958333,7.92,6.938776,17.16,75.2,97.7,A+
8,8,NaN,10.0,9.0,9.000000,10.000000,6.000000,8.00,7.000000,18.00,77.0,100.0,A+


## Convert to CSV

In [ ]:
# ai_final_df.to_csv('/content/drive/MyDrive/DSE_260_Capstone/Math 3B/capstone_gradebook.csv', index=True)
# no_ai_final_df.to_csv('/content/drive/MyDrive/DSE_260_Capstone/Math 3B/capstone_NO-AI_gradebook.csv', index=True)

#### Cleaned AI-USER Anonymized Gradebook saved in Drive as "capstone_gradebook.csv"
#### Cleaned NON-AI 'Anonymized Gradebook' saved in Drive as "capstone_NO-AI_gradebook.csv"

# DATA CLEANING - Grades-3B-May12-SP25
- CSV isGrades-3B-May12-SP25.csv
- Used to Get Tests (HW1-HW4) and Midterm

Cleaning:
- Ensure top line is header
- Remove NaNs
- Change features to shorter names (HW1 (964255) => HW1 Test)
- Change Student Entries from: Student 1 => 1 (For joining with AI and Non-AI students above)
- Add (Total Test) Score, GPA %, Letter Grade
- Export to CSV

In [ ]:
file_path_tests = '/content/drive/My Drive/DSE_260_Capstone/Math 3B/Grades-3B-May12-SP25.csv'
data_tests = pd.read_csv(file_path_tests)
data_tests.head(3)

,Student,Section,HW1 (964255),HW2 (965903),HW3 (967257),HW4 (968731),Probl Bank M1 (967258),03-31 (954343),04-02 (959106),04-04 (960143),...,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Seat Assignments Final Score,Seat Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score
0,Points Possible,NaN,45.0,45.0,45.0,45.0,45.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,(read only),(read only),(read only),(read only),(read only),(read only)
1,Student 1,MATH 3B - A04 [848369],NaN,NaN,42.0,NaN,45.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,75.48,75.48,60.9,60.9
2,Student 2,MATH 3B - A01 [848358],44.0,NaN,41.0,42.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,80.43,80.43,69.53,69.53


In [ ]:
singleHeader_tests_df = data_tests.iloc[1:]  # Remove the top row (Points Possible along row). Starts AFTER index 0 and later...
singleHeader_tests_df.head(3)

,Student,Section,HW1 (964255),HW2 (965903),HW3 (967257),HW4 (968731),Probl Bank M1 (967258),03-31 (954343),04-02 (959106),04-04 (960143),...,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Seat Assignments Final Score,Seat Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score
1,Student 1,MATH 3B - A04 [848369],NaN,NaN,42.0,NaN,45.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,75.48,75.48,60.9,60.9
2,Student 2,MATH 3B - A01 [848358],44.0,NaN,41.0,42.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,80.43,80.43,69.53,69.53
3,Student 3,MATH 3B - A03 [848368],45.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,87.18,87.18,7.81,7.81


## Shorten the Titles
- Ensure that the column to be joined (Student #) matches the feature from AI and Non-AI users
- Keep HW (10pt assignments) if valid int to replace 'calculated' values above for AI and Non-AI students. These are more up to date, correct scores, rather than means.
- Leave off Week 6 HW - 3.2 Precalc 2e (965987) and Week 7 HW - 3.3-3.4 Precalc 2e (967483)...Not Enough Scores
- Add Midterm 1

The following Dataframes end in "_tests" not "_df" as above. Prefix is the same.

The Gradebook shows single values for Homeworks 2.1-2.5. Since this is provided by the AI/NonAI testing, and too complex to multiply (also, values don't match), only uses TESTS here (HW1-4, and Midterm)

In [ ]:
renamed_tests = singleHeader_tests_df.rename(columns={
    'Student': 'Student #',  # Matches AI/Non-AI title
    'HW1 (964255)': 'HW 1 Test',
    'HW2 (965903)': 'HW 2 Test',
    'HW3 (967257)': 'HW 3 Test',
    'HW4 (968731)': 'HW 4 Test',
    'Midterm 1 (967259)': 'Midterm 1'

    # Comment All using shortcut: Cmd-/
    # 'Week 2 HW - Precalc 2e Section 1.1 (959893)': 'HW 1.1', # Week X HW" Matches AI/Non-AI titles
    # 'Week 2 HW - Precalc 2e Section 1.2 (959895)': 'HW 1.2',
    # 'Week 3 HW - 1.3 Precalc 2e (961458)': 'HW 1.3',
    # 'Week 3 HW - 1.5 Precalc 2e (961459)': 'HW 1.4',
    # 'Week 4 HW - 2.1 Precalc 2e (963053)': 'HW 2.1',
    # 'Week 4 HW - 2.2 Precalc 2e (963054)': 'HW 2.2',
    # 'Week 4 HW - 2.3 Precalc 2e (963055)': 'HW 2.3',
    # 'Week 5 HW - 1.5 CA / 2.5 P (964862)': 'HW 2.5',
    # 'Week 6 HW - 3.2 Precalc 2e (965987)': 'HW 3.2',

    })

In [ ]:
renamed_tests.head(3)

,Student #,Section,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Probl Bank M1 (967258),03-31 (954343),04-02 (959106),04-04 (960143),...,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Seat Assignments Final Score,Seat Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score
1,Student 1,MATH 3B - A04 [848369],NaN,NaN,42.0,NaN,45.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,75.48,75.48,60.9,60.9
2,Student 2,MATH 3B - A01 [848358],44.0,NaN,41.0,42.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,80.43,80.43,69.53,69.53
3,Student 3,MATH 3B - A03 [848368],45.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,87.18,87.18,7.81,7.81


In [ ]:
shortTitle_tests= pd.DataFrame(renamed_tests[['Student #',
    'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test', 'Midterm 1']])
shortTitle_tests.head(3)

,Student #,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1
1,Student 1,NaN,NaN,42.0,NaN,32.75
2,Student 2,44.0,NaN,41.0,42.0,37.25
3,Student 3,45.0,NaN,NaN,NaN,NaN


In [ ]:
shortTitle_tests.dtypes

,0
Student #,object
HW 1 Test,float64
HW 2 Test,float64
HW 3 Test,float64
HW 4 Test,float64
Midterm 1,float64


## Convert Strings to Ints

In [ ]:
# It looks like they ALL are FLOATS or INTS above => No need.
# This is how we would do it:
'''
assignment_tests = [
    'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test', 'Midterm 1']

shortTitle_tests[assignment_tests] = shortTitle_tests[assignment_tests].apply(pd.to_numeric, errors='coerce') # Converts to int if not None, else replaces with itself (should be blank)
'''

"\nassignment_tests = [\n    'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test', 'Midterm 1']\n\nshortTitle_tests[assignment_tests] = shortTitle_tests[assignment_tests].apply(pd.to_numeric, errors='coerce') # Converts to int if not None, else replaces with itself (should be blank)\n"

## Convert 'Student #' to "#"
- Removes "Student" from entry

In [ ]:
studentNum_tests= pd.DataFrame(shortTitle_tests)
studentNum_tests['Student #'] = studentNum_tests['Student #'].str.split(' ').str[1] # Uses 1 since 2nd element after 'Student' on space split
studentNum_tests.head(3)

,Student #,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1
1,1,NaN,NaN,42.0,NaN,32.75
2,2,44.0,NaN,41.0,42.0,37.25
3,3,45.0,NaN,NaN,NaN,NaN


In [ ]:
studentNum_tests.count()

,0
Student #,140
HW 1 Test,124
HW 2 Test,119
HW 3 Test,121
HW 4 Test,117
Midterm 1,132


## Drop NAs

In [ ]:
dropped_tests= pd.DataFrame(studentNum_tests)
dropped_tests.dropna(inplace=True)
dropped_tests.head(3)

,Student #,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1
4,4,42.75,34.0,25.5,40.0,42.25
5,5,45.00,39.0,41.5,39.0,47.25
6,6,42.00,40.5,42.5,38.0,12.25


In [ ]:
dropped_tests.count()

,0
Student #,103
HW 1 Test,103
HW 2 Test,103
HW 3 Test,103
HW 4 Test,103
Midterm 1,103


## Assign Test_Score, Test_GPA_Percent, and Test_Letter_Grade
- Add Midterm Weight = 50%, and Tests = 100%-Midterm Weight (~12.5% per HW Test)

In [ ]:
assignment_tests = [
    'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test', 'Midterm 1']

homework_tests = [
    'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test']

scored_tests = pd.DataFrame(dropped_tests)

scored_tests["HW Test Score"] = calculate_total_points(scored_tests, homework_tests) # Test scores HW 1 Test, HW 2 Test, ... HW4 Test

MIDTERM_WEIGHT = 0.6
TEST_WEIGHT = 1 - MIDTERM_WEIGHT

HW_TOTAL = 45 * 4
MIDTERM_TOTAL = 51

#scored_tests["Test Score"] = calculate_total_points(scored_tests, assignment_tests) # Recall assignment_tests: HW 1 Test, HW 2 Test, ... Midterm
scored_tests["Test GPA Percent"] = (((scored_tests["HW Test Score"] * TEST_WEIGHT / HW_TOTAL) + (scored_tests["Midterm 1"] * MIDTERM_WEIGHT / MIDTERM_TOTAL)) * 100).round(1)
scored_tests['Test Letter Grade'] = scored_tests['Test GPA Percent'].apply(assign_letter_grade)

# Drop the unused 'HW Test Score' - Won't be used again--only to discover final test score. Uncomment to see values:
scored_tests.drop(columns=['HW Test Score'], inplace=True)

scored_tests.tail(50)

,Student #,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade
77,77,45.0,43.0,44.25,45.0,44.00,91.1,A-
78,78,41.0,42.0,32.50,33.0,32.75,71.5,C-
79,79,43.0,33.0,0.00,0.0,45.00,69.8,D+
80,80,42.0,42.0,43.50,42.0,34.75,78.5,C+
81,81,44.0,41.0,38.50,34.5,45.00,88.1,B+
82,82,44.0,41.5,39.50,42.0,41.25,85.6,B
83,83,42.0,30.0,44.50,37.0,29.00,68.2,D+
85,85,42.0,44.0,41.00,35.0,38.25,81.0,B-
86,86,44.0,44.0,44.25,43.0,49.00,96.6,A
87,87,40.0,40.0,44.00,42.0,44.00,88.7,B+


In [ ]:
# assignment_tests = [
#     'HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test', 'Midterm 1']

# scored_tests = pd.DataFrame(dropped_tests)
# scored_tests["Test Score"] = calculate_total_points(scored_tests, assignment_tests) # Recall assignment_tests: HW 1 Test, HW 2 Test, ... Midterm
# scored_tests["Test GPA Percent"] = (scored_tests["Test Score"] / (45+45+45+45+51) * 100).round(1)
# scored_tests['Test Letter Grade'] = scored_tests['Test GPA Percent'].apply(assign_letter_grade)
# scored_tests.head(3)

## Replace Low (0.0) Scores
- Note that Student # 60, 79, 101, 121 have scores of 0 on at least 1 assignment
- These students likely dropped that test and retook it.
- Place average scores in these columns. These will have long float values => Round (1)

In [ ]:
no_low_tests = pd.DataFrame(scored_tests)

# List of homework test columns
homework_tests = ['HW 1 Test', 'HW 2 Test', 'HW 3 Test', 'HW 4 Test']

# For each homework column
for col in homework_tests:
    # Calculate mean excluding 0s
    non_zero_mean = no_low_tests.loc[no_low_tests[col] != 0, col].mean()
    # Replace 0s with the computed mean
    no_low_tests[col] = no_low_tests[col].replace(0.0, non_zero_mean).round(1)

no_low_tests.tail(30)

,Student #,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade
101,101,43.0,38.9,41.8,35.0,33.00,65.4,D
102,102,45.0,44.0,44.5,43.0,43.00,89.8,B+
106,106,42.0,28.0,42.0,43.0,35.50,76.2,C
107,107,43.0,44.0,43.0,40.0,48.00,94.2,A
108,108,41.5,42.0,44.5,43.0,46.00,92.1,A-
109,109,41.5,34.5,42.0,38.5,44.25,86.8,B
111,111,21.0,38.0,29.5,38.0,32.25,66.1,D
113,113,42.0,40.0,39.2,35.5,30.25,70.4,C-
114,114,42.0,45.0,44.5,42.0,24.75,67.7,D+
115,115,43.0,43.0,44.5,41.0,47.00,93.4,A


## Assign Clean TEST dataframe to final value "tests_df"

In [ ]:
tests_df = pd.DataFrame(no_low_tests)

# JOIN AI with Tests and Non-AI with Tests

In [ ]:
ai_df = pd.merge(ai_final_df, tests_df, on='Student #', how='inner')
ai_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW Letter Grade,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade
0,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,76.0,98.7,A+,42.8,34.0,25.5,40.0,42.25,81.3,B-
1,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,77.0,100.0,A+,45.0,39.0,41.5,39.0,47.25,92.1,A-
2,6,178739730,10.0,9.0,8.0,8.0,6.0,8.0,7.0,18.0,74.0,96.1,A,42.0,40.5,42.5,38.0,12.25,50.6,F


In [ ]:
len(ai_df)

64

In [ ]:
none_df = pd.merge(no_ai_final_df, tests_df, on='Student #', how='inner')
none_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW Letter Grade,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade
0,8,NaN,10.0,9.0,9.0,10.0,6.0,8.0,7.000000,18.00,77.0,100.0,A+,45.0,45.0,44.5,43.0,40.75,87.4,B+
1,16,NaN,10.0,9.0,9.0,7.0,6.0,7.0,6.938776,17.16,72.1,93.6,A,44.0,43.0,43.0,13.0,41.00,80.0,B-
2,19,NaN,10.0,9.0,9.0,10.0,6.0,8.0,7.000000,18.00,77.0,100.0,A+,45.0,43.5,43.0,36.0,50.00,96.0,A


In [ ]:
len(none_df)

39

In [ ]:
ai_df.describe()

,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,9.933066,8.983275,8.871347,9.751015,5.935471,7.882711,6.949747,17.801098,76.107812,98.840625,40.895313,38.998438,40.982813,39.117188,38.921875,81.203125
std,0.393198,0.124918,0.417248,0.524387,0.301897,0.440149,0.212695,0.498339,1.953694,2.540261,5.393689,5.630853,4.890144,3.678274,9.848644,13.421955
min,7.000000,8.000000,7.000000,8.000000,4.000000,5.000000,6.000000,16.000000,67.000000,87.000000,20.000000,17.000000,15.000000,30.000000,5.250000,34.200000
25%,10.000000,9.000000,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,76.000000,98.700000,40.000000,38.000000,39.875000,35.750000,34.187500,74.525000
50%,10.000000,9.000000,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,42.000000,40.000000,42.350000,40.000000,40.250000,83.100000
75%,10.000000,9.000000,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,44.000000,43.000000,44.200000,42.000000,47.250000,92.150000
max,10.000000,9.000000,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,45.000000,45.000000,45.000000,45.000000,51.000000,98.400000


In [ ]:
none_df.describe()

,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,HW Score,HW GPA_Percent,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent
count,39.0,39.0,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000
mean,10.0,9.0,8.555207,9.500785,5.945513,7.895385,6.945578,17.645128,75.487179,98.030769,40.705128,38.830769,40.815385,37.800000,39.192308,80.356410
std,0.0,0.0,1.712221,1.446419,0.222984,0.306919,0.223135,1.178694,3.498885,4.550370,7.569605,7.718132,5.739500,7.253239,10.728170,17.140607
min,10.0,9.0,0.000000,3.000000,5.000000,7.000000,6.000000,11.000000,60.000000,77.900000,9.000000,7.000000,13.000000,13.000000,9.500000,23.800000
25%,10.0,9.0,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,76.000000,98.700000,40.750000,37.000000,40.000000,36.500000,35.250000,77.600000
50%,10.0,9.0,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,43.000000,40.000000,43.000000,40.000000,44.000000,86.800000
75%,10.0,9.0,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,45.000000,43.000000,44.200000,42.000000,47.000000,92.350000
max,10.0,9.0,9.000000,10.000000,6.000000,8.000000,7.000000,18.000000,77.000000,100.000000,45.000000,45.000000,45.000000,43.000000,50.000000,96.000000


## Convert to CSV

In [ ]:
# tests_df.to_csv('/content/drive/MyDrive/DSE_260_Capstone/Math 3B/capstone_tests.csv', index=True)
ai_df.to_csv('/content/drive/MyDrive/DSE_260_Capstone/Math 3B/capstone_ai.csv', index=True)
none_df.to_csv('/content/drive/MyDrive/DSE_260_Capstone/Math 3B/capstone_none.csv', index=True)

# DATA CLEANING - Raw Data Coversations

In [ ]:
file_path_conversations = '/content/drive/My Drive/DSE_260_Capstone/Math 3B/capstone_rawdata_conversations.csv'
# DOES NOT WORK. Has unreadable long dash - data_conversations = pd.read_csv(file_path_conversations
# DOES NOT WORK. Has "Unnamed 6-26 - data_conversations = pd.read_csv(file_path_conversations, encoding='cp1252') # Use encoding to bypass long dash "—" (vs '-'), invalid UTF-8
data_conversations = pd.read_csv(file_path_conversations, encoding='ISO-8859-1')
data_conversations.head(3)

,Order,ChatID,Vocareum ID,Timestamp,Date/Time,Message,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,120,2892,178742970,1743621501,4/2/2025 19:18:21,how would you rewrite a function?,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!
1,121,2892,0,1743621509,4/2/2025 19:18:29,"**_Key Idea:_** \n When rewriting a function,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,122,2892,178742970,1743621621,4/2/2025 19:20:21,should we divide it,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Lots of useless 'Unnamed'.
- VocareumID# is the student
- Blanks IDs (the ones with 0's) are the AI's response.
- Better to read it correctly

In [ ]:
data_conversations = pd.read_csv(
    file_path_conversations,
    encoding='cp1252',
    quotechar='"',
    on_bad_lines='skip'  # skip weird rows, or use 'warn' to debug
)

In [ ]:
data_conversations.head(3)

,Order,ChatID,Vocareum ID,Timestamp,Date/Time,Message,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,120,2892,178742970,1743621501,4/2/2025 19:18:21,how would you rewrite a function?,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!
1,121,2892,0,1743621509,4/2/2025 19:18:29,"**_Key Idea:_** \n When rewriting a function,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,122,2892,178742970,1743621621,4/2/2025 19:20:21,should we divide it,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop extraneous unnamed columns

In [ ]:
data_conversations = data_conversations.loc[:, ~data_conversations.columns.str.contains('^Unnamed')]
data_conversations.head(3)

,Order,ChatID,Vocareum ID,Timestamp,Date/Time,Message
0,120,2892,178742970,1743621501,4/2/2025 19:18:21,how would you rewrite a function?
1,121,2892,0,1743621509,4/2/2025 19:18:29,"**_Key Idea:_** \n When rewriting a function,..."
2,122,2892,178742970,1743621621,4/2/2025 19:20:21,should we divide it


### Label who is AI vs. Student

In [ ]:
data_conversations['SenderType'] = data_conversations['Vocareum ID'].apply(
    lambda x: 'AI' if x == 0 else 'Student'
)
data_conversations.head(3)

,Order,ChatID,Vocareum ID,Timestamp,Date/Time,Message,SenderType
0,120,2892,178742970,1743621501,4/2/2025 19:18:21,how would you rewrite a function?,Student
1,121,2892,0,1743621509,4/2/2025 19:18:29,"**_Key Idea:_** \n When rewriting a function,...",AI
2,122,2892,178742970,1743621621,4/2/2025 19:20:21,should we divide it,Student


### Check to ensure Vocareum ID is read as STRINGS not FLOATS
- This will mess with the conversion when grouping or summing

In [ ]:
data_conversations['Vocareum ID'].dtype

dtype('int64')

In [ ]:
# Convert to Strings if necessary
data_conversations['Vocareum ID'] = data_conversations['Vocareum ID'].astype(str).str.strip()
data_conversations['Vocareum ID'].dtype

dtype('O')

In [ ]:
data_conversations.dtypes

,0
Order,int64
ChatID,int64
Vocareum ID,object
Timestamp,int64
Date/Time,object
Message,object
SenderType,object


In [ ]:
data_conversations['Vocareum ID'].unique()

array(['178742970', '0', '179072685', '179072730', '178740990',
       '178744140', '179064990', '178743690', '178740135', '179064720',
       '179072640', '178739910', '179072370', '179072460', '178743555',
       '178739685', '178742160', '178744635', '178744590', '178740405',
       '179072820', '178743915', '178741980', '178743240', '178739370',
       '178742835', '178742340', '178740540', '153789210', '153790155',
       '179361540', '178743375', '179457750', '178743825', '179368110',
       '179357400', '178743060', '178739820', '178742475', '178741935',
       '178740945', '179064675', '179467830', '178741530', '178741845',
       '179073090', '178744365', '178743960', '178743600', '178743105',
       '178741305', '179468775', '178744410', '178910595', '178742565',
       '179072775', '179445105', '178741350', '179468370', '178741080',
       '178744095', '178740180', '179064765', '178743510', '179066835',
       '178740090', '178739865', '178742925', '179073045', '178744185',


In [ ]:
data_conversations['Vocareum ID'].value_counts().head(10)

,count
Vocareum ID,
0,1975
179072820,177
178742970,137
178741845,117
178744410,95
178741080,80
179072775,73
178742835,67
178739865,62


### Count messages by student

In [ ]:
student_msg_counts_df = (
    data_conversations[data_conversations['SenderType'] == 'Student']
    .groupby('Vocareum ID')['Message']
    .count()
    .reset_index(name='StudentMessageCount')
)
student_msg_counts_df.head(13)

,Vocareum ID,StudentMessageCount
0,153789210,16
1,153789525,4
2,153790155,22
3,161451000,1
4,178739325,1
5,178739370,35
6,178739415,3
7,178739595,1
8,178739685,8
9,178739730,11


### Count Messages by AI

In [ ]:
grouped_msgs_df = (
    data_conversations
    .groupby(['ChatID', 'SenderType'])['Message']
    .apply(lambda msgs: ' '.join(str(m) for m in msgs))
    .reset_index()
)
grouped_msgs_df.head(13)

,ChatID,SenderType,Message
0,2892,AI,"**_Key Idea:_** \n When rewriting a function,..."
1,2892,Student,how would you rewrite a function? should we di...
2,2893,AI,**? Absolutely!** \n When you substitute \(y=...
3,2893,Student,can I combine like terms? once I combine the p...
4,2894,AI,**_Let's Explore This Together!_**\n \n\n - **...
5,2894,Student,"would the answer be (0,-6) 0 so would i put (0..."
6,2895,AI,"**Absolutely!** \n *To think about it*, you'l..."
7,2895,Student,would i substitute the 4? would i square root ...
8,2896,AI,**Here's an Elegant Hint:** \n When reading a...
9,2896,Student,Help me understand the following concept betwe...


In [ ]:
invalid_ids = data_conversations[~data_conversations['Vocareum ID'].str.match(r'^17\d{7}$')]['Vocareum ID'].unique()
print(invalid_ids)

['0' '153789210' '153790155' '180406530' '161451000' '180417870'
 '153789525']


### Now, we have 2 distinct Dataframes: Conversation with # Msgs and ChatIDs.
- Since Conversations with #'s are distinct by Vocareum ID, we can add it to our AI df to lengthen it and give it meaning.
- We should add the same column to our NO_AI df, to match.

- Our other ChatID df has multiple VocareumIDs per many ChatIDs. Therefore, we keep it separate.
- However, we CAN add the VocareumID to the end to match (though it will not be distinct). Will have the same Vocareum IDs for multiple ChatIDs for long conversations.

### Merge AI Dataframe with Total # of Student/AI conversations
- ai_chat_df is the prior ai_final_df with StudentMessageCount at end

In [ ]:
ai_chatTotal_df = ai_df.merge(
    student_msg_counts_df,
    on='Vocareum ID',
    how='left'
)

In [ ]:
ai_chatTotal_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,...,HW GPA_Percent,HW Letter Grade,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade,StudentMessageCount
0,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,...,98.7,A+,42.8,34.0,25.5,40.0,42.25,81.3,B-,35
1,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,...,100.0,A+,45.0,39.0,41.5,39.0,47.25,92.1,A-,19
2,6,178739730,10.0,9.0,8.0,8.0,6.0,8.0,7.0,18.0,...,96.1,A,42.0,40.5,42.5,38.0,12.25,50.6,F,11


### Also add "Student Message Count" to the Non-AI
- It will just be 0, but for consistency when using Tableau

In [ ]:
no_ai_chatTotal_df = pd.DataFrame(none_df)
no_ai_chatTotal_df['StudentMessageCount'] = 0
no_ai_chatTotal_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,...,HW GPA_Percent,HW Letter Grade,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade,StudentMessageCount
0,8,NaN,10.0,9.0,9.0,10.0,6.0,8.0,7.000000,18.00,...,100.0,A+,45.0,45.0,44.5,43.0,40.75,87.4,B+,0
1,16,NaN,10.0,9.0,9.0,7.0,6.0,7.0,6.938776,17.16,...,93.6,A,44.0,43.0,43.0,13.0,41.00,80.0,B-,0
2,19,NaN,10.0,9.0,9.0,10.0,6.0,8.0,7.000000,18.00,...,100.0,A+,45.0,43.5,43.0,36.0,50.00,96.0,A,0


### Add the Vocareum ID to Chat ID dataframe to indicate student

In [ ]:
# ACK!! THIS ISN'T NECESSARY
# The VOCAREUM ID is already there!

# chat_df = grouped_msgs_df.merge(
#     data_conversations[['ChatID', 'Vocareum ID']],
#     on='ChatID',
#     how='left'
# )
# chat_df.head(6)

## Convert To CSVs

In [ ]:
ai_chatTotal_df.to_csv('/content/drive/My Drive/DSE_260_Capstone/Math 3B/capstone_ai_chatTotal.csv', index=False)
no_ai_chatTotal_df.to_csv('/content/drive/My Drive/DSE_260_Capstone/Math 3B/capstone_no_ai_chatTotal.csv', index=False)
# chat_df.to_csv('/content/drive/My Drive/DSE_260_Capstone/Math 3B/capstone_chat_individual.csv', index=False)

In [ ]:
ai_chatTotal_df.head(3)

,Student #,Vocareum ID,HW 1.1,HW 1.2,HW 1.3,HW 1.4,HW 2.1,HW 2.2,HW 2.3,HW 2.5,...,HW GPA_Percent,HW Letter Grade,HW 1 Test,HW 2 Test,HW 3 Test,HW 4 Test,Midterm 1,Test GPA Percent,Test Letter Grade,StudentMessageCount
0,4,178739370,10.0,9.0,9.0,10.0,6.0,8.0,6.0,18.0,...,98.7,A+,42.8,34.0,25.5,40.0,42.25,81.3,B-,35
1,5,179357400,10.0,9.0,9.0,10.0,6.0,8.0,7.0,18.0,...,100.0,A+,45.0,39.0,41.5,39.0,47.25,92.1,A-,19
2,6,178739730,10.0,9.0,8.0,8.0,6.0,8.0,7.0,18.0,...,96.1,A,42.0,40.5,42.5,38.0,12.25,50.6,F,11
